In [1]:
import numpy as np
import pandas as pd
import os, re, random, gc
import pydicom as pdc
from utils.datahandling_utils import createImageArray


In [2]:
SEED = 41
BATCH_SIZE = 64
random.seed(SEED)

In [3]:
label_path = "D:/AIFs/AIFs/durable/BorrSci_MR_Data/Output"
image_path = "D:/iCAT_IMAGES"

In [5]:
data = pd.read_csv("data_indices.csv", ",", index_col = 0)
data = data.sort_values(["Patient", "Volume", "Slice"], ignore_index=True)

C:\Users\svenalrik\AppData\Local\Temp\ipykernel_10492\4254050036.py:1: FutureWarning: In a future version of pandas all arguments of read_csv except for the argument 'filepath_or_buffer' will be keyword-only.
  data = pd.read_csv("data_indices.csv", ",", index_col = 0)


In [6]:
# patient = data.loc[(data['Patient']==14)]
# details = []
# for pos in range(patient.shape[0]):
#     dcm = pdc.read_file(patient.iloc[pos]['ImagePath'])
#     details.append([dcm.AcquisitionNumber, dcm.SliceLocation])

In [7]:
# pd = pd.DataFrame(details, columns=["ImagePos", "InstanceNumber", "AcqNumber", "SeriesNumber", "SliceLocation"])

In [8]:
# acquisition number = volume, slice location = slice number
# np.unique(pd['SliceLocation'])


In [11]:
def getPaths(path):
    paths = []
    dir_list = []
    for root, dirs, files in os.walk(path):
        if len(dirs) < 1:
            for i in range(len(files)):
                path = '/'
                path_list = (root+'/'+files[i]).split('\\')
                path = path.join(path_list)
                paths.append(path)
        else:
            dir_list.append(dirs)
    if len(dir_list) < 1:
        return paths, dir_list
    return paths, dir_list[0]

# Må tilpasses slik at sailor datasettet også kan indekseres.
def createImageIndexCSV(path, force=False):
    result_file_path = "data_indices.csv"
    if os.path.isfile(result_file_path) and not force:
        return pd.read_csv(result_file_path, delimiter=',', index_col=0)
    result_frame = []
    paths, dirs = getPaths(path)
    for patient in dirs:
        patient_data = []
        slice_positions = []
        patient_paths = get_list_of_paths(paths, patient)
        for image_path in patient_paths:
            dcm = pdc.read_file(image_path)
            patient_data.append([patient, int(dcm.AcquisitionNumber), dcm.SliceLocation, image_path])
            slice_positions.append(dcm.SliceLocation)
        slice_positions = np.unique(np.array(slice_positions))
        for data in patient_data:
            slice_num = np.where(slice_positions == data[2])
            result_frame.append([data[0], data[1], int(slice_num[0]) + 1, data[3]])
    df = pd.DataFrame(result_frame, columns= ["Patient", "Volume", "Slice", "ImagePath"])
    df.to_csv(result_file_path)
    return df

def get_list_of_paths(paths, patient):
    patient_filter = re.compile("(.*?)\/" + patient + "\/.*")
    patient_paths = list(filter(patient_filter.match, paths))
    return patient_paths

def get_curr_slice_and_volume(image, num_slices = 48):
    split_list = image.split("_")
    number = int(split_list[-1].split(".")[0])
    if (number <= num_slices):
        slice_num = number % (num_slices + 1)
        volume_num = number // (num_slices + 1)
    elif number % num_slices == 0:
        slice_num = num_slices
        volume_num = (number // num_slices) - 1
    else:
        slice_num = number % num_slices
        volume_num = number // num_slices 
    return slice_num, volume_num+1

In [12]:
createImageIndexCSV(image_path, force=True)

,Patient,Volume,Slice,ImagePath
0,001,1,[1],D:/iCAT_IMAGES/001/001_20_1.dcm
1,001,1,[10],D:/iCAT_IMAGES/001/001_20_10.dcm
2,001,3,[4],D:/iCAT_IMAGES/001/001_20_100.dcm
3,001,21,[40],D:/iCAT_IMAGES/001/001_20_1000.dcm
4,001,21,[41],D:/iCAT_IMAGES/001/001_20_1001.dcm
...,...,...,...,...
268794,070,21,[35],D:/iCAT_IMAGES/070/006_18_995.dcm
268795,070,21,[36],D:/iCAT_IMAGES/070/006_18_996.dcm
268796,070,21,[37],D:/iCAT_IMAGES/070/006_18_997.dcm
268797,070,21,[38],D:/iCAT_IMAGES/070/006_18_998.dcm


In [26]:
# import matplotlib.pyplot as plt
# max_slopes = np.zeros((array.shape[1], array.shape[2]))
# for row in range(array.shape[1]):
#     for col in range(array.shape[2]):
#         slopes = np.zeros(array.shape[0]-1)
#         for i in range(array.shape[0]-1):
#             slope = array[i+1, row, col] - array[i, row, col]
#             slopes[i] = slope
#         max_slopes[row, col] = np.max(slopes)

In [10]:
# _ = get_AIF_KMeans(slices, seed=SEED, visualize=True)

In [11]:
# class IndexTracker:
#     def __init__(self, ax, X):
#         self.ax = ax
#         ax.set_title('use scroll wheel to navigate images')

#         self.X = X
#         self.slices, rows, cols = X.shape
#         self.ind = self.slices//2

#         self.im = ax.imshow(self.X[self.ind, :, :])
#         self.update()

#     def on_scroll(self, event):
#         print("%s %s" % (event.button, event.step))
#         if event.button == 'up':
#             self.ind = (self.ind + 1) % self.slices
#         else:
#             self.ind = (self.ind - 1) % self.slices
#         self.update()

#     def update(self):
#         self.im.set_data(self.X[self.ind, :, :])
#         self.ax.set_ylabel('slice %s' % self.ind)
#         self.im.axes.figure.canvas.draw()



In [12]:
# %matplotlib qt
# fig, ax = plt.subplots(1, 1)

# tracker = IndexTracker(ax, slices)


# fig.canvas.mpl_connect('scroll_event', tracker.on_scroll)
# plt.show()